In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import hddm 
from patsy import dmatrix 
print(hddm.__version__)

%matplotlib inline

0.6.0


/Users/Krista/anaconda/envs/hddm/lib/python3.5/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


## Pre-processing

In [2]:
!pwd

/Users/Krista/Dropbox/volatileValues/simple_rt_experiment_probabilityC/analysis


In [3]:
data = hddm.load_csv('./cavanagh_theta_nn.csv')

In [4]:
data.head(10)

,subj_idx,stim,rt,response,theta,dbs,conf
0,0,LL,1.210,1.0,0.656275,1,HC
1,0,WL,1.630,1.0,-0.327889,1,LC
2,0,WW,1.030,1.0,-0.480285,1,HC
3,0,WL,2.770,1.0,1.927427,1,LC
4,0,WW,1.140,0.0,-0.213236,1,HC
5,0,WL,1.150,1.0,-0.436204,1,LC
6,0,LL,2.000,1.0,-0.274479,1,HC
7,0,WL,1.040,0.0,0.666957,1,LC
8,0,WW,0.857,1.0,0.118617,1,HC
9,0,WL,1.500,0.0,0.823626,1,LC


In [5]:
data = hddm.utils.flip_errors(data)

## Within-subjects regression 

#### Treatment specifies intercept (comparison condition) 
#### C(x) specifies categorical predictor 

##### (design matrix will auto-code dummy variables)

In [6]:
dmatrix("C(stim,Treatment('WL'))", data.head(10))

DesignMatrix with shape (10, 3)
  Intercept  C(stim, Treatment('WL'))[T.LL]  C(stim, Treatment('WL'))[T.WW]
          1                               1                               0
          1                               0                               0
          1                               0                               1
          1                               0                               0
          1                               0                               1
          1                               0                               0
          1                               1                               0
          1                               0                               0
          1                               0                               1
          1                               0                               0
  Terms:
    'Intercept' (column 0)
    "C(stim, Treatment('WL'))" (columns 1:3)

#### trialwise response ~ categorical predictor, intercept condition 

In [7]:
v_reg_model = "v ~ C(stim, Treatment('WL'))"
a_reg_model = "a ~ C(stim, Treatment('WL'))"
av_reg_model = [v_reg_model, a_reg_model]

#null: condition has no effect on drift 
null_v_reg_model = "v~1"
#null: condition has no effect on bound 
null_a_reg_model = "a~1"
#null: condition has no effect on bound/drift simultaneously estimated
null_av_reg_model = [null_v_reg_model, null_a_reg_model]

In [33]:
v_within_subj = hddm.HDDMRegressor(data, v_reg_model)
v_within_subj.sample(5,burn=1, dbname='traces.db', db='pickle')

Adding these covariates:
['v_Intercept', "v_C(stim, Treatment('WL'))[T.LL]", "v_C(stim, Treatment('WL'))[T.WW]"]
 [--------------------120%---------------------] 6 of 5 complete in 8.6 sec

In [ ]:
a_within_subj = hddm.HDDMRegressor(data, a_reg_model)
a_within_subj.sample(5000,burn=200)

In [ ]:
av_within_subj = hddm.HDDMRegressor(data, av_reg_model)
av_within_subj.sample(5000,burn=200)

#### null models

In [ ]:
null_v_within_subj = hddm.HDDMRegressor(data, null_v_reg_model)
null_v_within_subj.sample(5000,burn=200)

In [ ]:
null_a_within_subj = hddm.HDDMRegressor(data, null_a_reg_model)
null_a_within_subj.sample(5000,burn=200)

In [ ]:
null_av_within_subj = hddm.HDDMRegressor(data, null_av_reg_model)
null_av_within_subj.sample(5000,burn=200)

#### plot within-subjects effects 

In [ ]:
v_WL, v_LL, v_WW = v_within_subj.nodes_db.ix[["v_Intercept", 
                                              "v_C(stim, Treatment('WL'))[T.LL]", 
                                              "v_C(stim, Treatment('WL'))[T.WW]"], 'node']
hddm.analyze.plot_posterior_nodes([v_WL, v_LL, v_WW])
plt.xlabel('drift-rate')
plt.ylabel('Posterior probability')
plt.title('Group mean posteriors of within-subject drift-rate effects.')
# plt.savefig('hddm_demo_fig_07.pdf')

In [ ]:
a_WL, a_LL, a_WW = a_within_subj.nodes_db.ix[["a_Intercept", 
                                              "a_C(stim, Treatment('WL'))[T.LL]", 
                                              "a_C(stim, Treatment('WL'))[T.WW]"], 'node']
hddm.analyze.plot_posterior_nodes([a_WL, a_LL, a_WW])
plt.xlabel('bound')
plt.ylabel('Posterior probability')
plt.title('Group mean posteriors of within-subject bound effects.')
# plt.savefig('hddm_demo_fig_07.pdf')

#### extract DIC score

In [ ]:
v_within_subj.dic, a_within_subj.dic, av_within_subj.dic

In [ ]:

plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['axes.titlesize'] = 25
plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30
plt.rcParams['ytick.labelsize'] = 30

names = [r'$a + v$',r'$a$',r'$v$']
dics = [v_within_subj.dic,  a_within_subj.dic, av_within_subj.dic]
bars = plt.barh(names,dics, alpha=0.5)
plt.xlabel('DIC score')

bars[0].set_color('r')
bars[1].set_color('blue')
bars[2].set_color('forestgreen')

plt.axvline(av_within_subj.dic, color='k', alpha=0.5, linestyle = '--')

#### all stats

In [ ]:
# v_within_subj.print_stats()

In [36]:
v_within_subj.save('ex_a_reg' )

In [38]:
v_within_subj.load_db('ex_a_reg') 

DatabaseError: file is not a database